In [15]:
import pandas as pd
import openpyxl
import numpy as np
from openpyxl import load_workbook
import yfinance as yf
etfs = {
    "Energy": "XLE",
    "Materials": "XLB",
    "Industrials": "XLI",
    "Utilities": "XLU",  
    "Healthcare": "XLV",
    "Financials": "XLF",
    "Consumer Discretionary": "XLY",
    "Consumer Staples": "XLP",
    "Information Technology": "XLK",
    "Communication Services": "XLC",
    "Real Estate": "XLRE"
}
column_order = [
    "Ticker",  

    # Performance (40%) #
    "1Y Return",
    "3Y Avg Return",
    "6M Price Momentum",
    "Dividend Yield",

    # Risk + Volatility (27.5%) #
    "3Y Std Dev",
    "Beta",
    "Max Drawdown",
    "Sharpe Ratio",

    # Liquidity + Efficiency (20%) #
    "Volume",
    "Bid Ask Spread",
    
    # Strength (12.5%) #
    "Rsi",
    "Rsi vs SPY"
]

In [16]:
# Pulls 'easy' data, aka data that doesn't need to be calculated #
def pull_ez_data(tick):
    ticka = yf.Ticker(tick)
    info = ticka.info
    one_year_data = ticka.history(period="1y")
    one_year_return = (one_year_data["Close"].iloc[-1] / one_year_data["Close"].iloc[0] - 1) * 100  # Percentage return
    three_year_data = ticka.history(period="3y")
    three_year_return = (three_year_data["Close"].iloc[-1] / three_year_data["Close"].iloc[0] - 1) * 100  # Percentage return
    annualized_return = ((1 + (three_year_return / 100)) ** (1/3) - 1) * 100  # Annualized return
    dividend_data = ticka.dividends
    total_dividends = dividend_data.sum()  # Sum of dividends in the past year
    avg_price = one_year_data["Close"].mean()  # Average closing price for the past year
    dividend_yield = (total_dividends / avg_price) * 100  # Percentage yield
    return {
        "1-Year Total Return (%)": one_year_return,
        "3-Year Annualized Return (%)": annualized_return,
        "Dividend Yield (%)": dividend_yield,
    }


In [17]:
# Calculates 6 month price momentum #
def six_month(tick):
    ticka = yf.Ticker(tick)
    data = ticka.history(period="6mo")
    
    most_recent_close = data["Close"].iloc[-1]
    
    six_months_ago_close = data["Close"].iloc[0]
    
    price_momentum = ((most_recent_close / six_months_ago_close) - 1) * 100
    
    return price_momentum

In [18]:
# Calculates beta using (covariance / variance) #
def get_beta(ticker, benchmark="^GSPC", years=3):
    try:
        data = yf.download([ticker, benchmark], period=f"{years}y")["Close"]
        data = data.ffill()
        returns = data.pct_change().dropna()
        covariance = np.cov(returns[ticker], returns[benchmark])[0][1]
        benchmark_variance = np.var(returns[benchmark])
        beta = covariance / benchmark_variance
        return round(beta, 2)
    except Exception as e:
        print(f"Error calculating Beta: {e}")
        return None


In [19]:
# Calculates Standard Deviation of price over 3Y period #
def threeyr_stdev(tick):
    ticka = yf.Ticker(tick)
    data = ticka.history(period="3y")

    data['Close'] = data['Close'].ffill()
     
    data['Daily Return'] = data['Close'].pct_change().dropna()
    
    std_deviation = np.std(data['Daily Return'])
    
    return std_deviation

In [20]:
# Calculates Maximum Drawdown #
def max_drawdown(tick):
    ticka = yf.Ticker(tick)
    hist = ticka.history(period="3y")["Close"]
    
    cum_max = hist.cummax()  
    drawdown = (hist - cum_max) / cum_max  
    max_dd = drawdown.min() * 100  # Convert to percentage
    
    return max_dd

In [21]:
# Calculates Sharpe Ratio #
def h(tick):
    ticka = yf.Ticker(tick)
    hist = ticka.history(period="3y")["Close"]
    treasury = yf.Ticker("^IRX")  
    risk_free_rate = treasury.history(period="1d")["Close"].iloc[-1] / 100
    
    hist = hist.ffill()
    
    returns = hist.pct_change().dropna()
 
    excess_returns = returns - (risk_free_rate / 252)
  
    sharpe = (excess_returns.mean() / returns.std()) * (252 ** 0.5)  
    
    return sharpe

In [22]:
# Calculates daily volume over past 30 days #
def month_avg(tick):
    ticka = yf.Ticker(tick)
    volume_data = ticka.history(period="30d")["Volume"]
    avg_volume = volume_data.mean()  
    return avg_volume

In [23]:
# Bid Ask Spread (most recent) #
def bidask_spread(tick):
    tick_data = yf.Ticker(tick)
    
    hist_data = tick_data.history(period="1d")
    
    high_price = hist_data['High'].iloc[0]
    low_price = hist_data['Low'].iloc[0]

    spread = ((high_price - low_price) / ((high_price + low_price) / 2)) * 100
    return spread

In [24]:
# Calculates RSI (past 2 weeks) #
def rsi(tick, period=14):
    ticker = yf.Ticker(tick)
    hist = ticker.history(period="1y")
    delta = hist["Close"].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    avg_loss = avg_loss.replace(0, 0.0001)
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.iloc[-1]

In [25]:
# Checks RSI against $SPY #
def rsi_vs_spy(tick):
    ticker_rsi = rsi(tick)  
    sp500_rsi = rsi("^GSPC")  
    
    return ticker_rsi - sp500_rsi 


In [26]:
# Seperate normalization functions for values that indicate success at higher or lower values #
def normalize_hi(value, data):
    ma = max(data)
    mi = min(data)
    return (value - mi) / (ma - mi)

def normalize_lo(value, data):
    ma = max(data)
    mi = min(data)
    return (ma - value) / (ma - mi)

In [27]:
#Creates pandas dataframe for the orignial data #
raw_data=[]
for sector in etfs:
    datadict={}
    t=etfs[sector]
    datadict['Ticker']=etfs[sector]
    datadict['Rsi']=rsi(t)
    datadict['Rsi vs SPY']=rsi_vs_spy(t)
    datadict['Bid Ask Spread']=bidask_spread(t)
    datadict['Volume']=month_avg(t)
    datadict['Sharpe Ratio']=h(t)
    datadict['Max Drawdown']=max_drawdown(t)
    datadict['3Y Std Dev']=threeyr_stdev(t)
    datadict['Beta']=get_beta(t)
    datadict['6M Price Momentum']=six_month(t)
    last_three=pull_ez_data(t)
    datadict['1Y Return']=last_three['1-Year Total Return (%)']
    datadict['Dividend Yield']=last_three['Dividend Yield (%)']
    datadict['3Y Avg Return']=last_three['3-Year Annualized Return (%)']
    result = {key: datadict[key] for key in column_order}
    raw_data.append(result)
raw_df=pd.DataFrame(raw_data)

[                       0%                       ]

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed


In [29]:
rsi_vals = []
rsi_v_spy_vals = []
spread_vals = []
vol_vals = []
sharpe_vals = []
drawdown_vals = []
std_devs = []
beta_vals = []
six_m_vals = []
threeyr_avg_vals=[]
oneyr_vals = []
dividend_vals = []

# Collect the data to normalize
for datadict in raw_data:
    rsi_vals.append(datadict['Rsi'])
    rsi_v_spy_vals.append(datadict['Rsi vs SPY'])
    spread_vals.append(datadict['Bid Ask Spread'])
    vol_vals.append(datadict['Volume'])
    sharpe_vals.append(datadict['Sharpe Ratio'])
    drawdown_vals.append(datadict['Max Drawdown'])
    std_devs.append(datadict['3Y Std Dev'])
    beta_vals.append(datadict['Beta'])
    six_m_vals.append(datadict['6M Price Momentum'])
    oneyr_vals.append(datadict['1Y Return'])
    dividend_vals.append(datadict['Dividend Yield'])
    threeyr_avg_vals.append(datadict['3Y Avg Return'])

normalized_data = []

# Normalizes Data and puts it into pandas dataframe #
for datadict in raw_data:
    normalized_datadict = {}
    
    normalized_datadict['Ticker'] = datadict['Ticker']
    # Performance #
    normalized_datadict['1Y Return'] = normalize_hi(datadict['1Y Return'], oneyr_vals)  
    normalized_datadict['3Y Avg Return'] = normalize_hi(datadict['3Y Avg Return'], threeyr_avg_vals)  
    normalized_datadict['6M Price Momentum'] = normalize_hi(datadict['6M Price Momentum'], six_m_vals)  
    normalized_datadict['Dividend Yield'] = normalize_hi(datadict['Dividend Yield'], dividend_vals)  
    
    # Risk + Volatilty #
    normalized_datadict['3Y Std Dev'] = normalize_lo(datadict['3Y Std Dev'], std_devs)  
    normalized_datadict['Beta'] = normalize_hi(datadict['Beta'], beta_vals)  
    normalized_datadict['Max Drawdown'] = normalize_lo(datadict['Max Drawdown'], drawdown_vals)  
    normalized_datadict['Sharpe Ratio'] = normalize_hi(datadict['Sharpe Ratio'], sharpe_vals)  
    
    # Liquidity + Efficiency #
    normalized_datadict['Volume'] = normalize_hi(datadict['Volume'], vol_vals)  
    normalized_datadict['Bid Ask Spread'] = normalize_lo(datadict['Bid Ask Spread'], spread_vals)  
    
    # RSI #
    normalized_datadict['Rsi'] = normalize_lo(datadict['Rsi'], rsi_vals)  
    normalized_datadict['RSI vs SPY'] = normalize_hi(datadict['Rsi vs SPY'], rsi_v_spy_vals)
    normalized_data.append(normalized_datadict)

    normalized_df = pd.DataFrame(normalized_data)


file_path = "/Users/ericramos/OneDrive/Documents/SPDR ETF Tracker.xlsx"
wb = load_workbook(file_path)

orig_ws = wb["Original Data"]
normal_ws = wb["Normalized Data"]

for r_idx, row in enumerate(normalized_df.itertuples(index=False, name=None), start=3):  
    for c_idx, value in enumerate(row, start=2):
        normal_ws.cell(row=r_idx, column=c_idx, value=value)

for r_idx, row in enumerate(raw_data, start=3):  
    for c_idx, (col_name, value) in enumerate(row.items(), start=2):  
        orig_ws.cell(row=r_idx, column=c_idx, value=value)

wb.save(file_path)

